In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
from matplotlib import pyplot as plt
import scvelo as scv
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from scipy import io

In [ ]:
oviendo = sc.read_h5ad('/home/j87832lw/mounting/oviduct/data/alldatasetclustering.h5ad')

In [ ]:
sc.pl.umap(oviendo, color = 'Cell_Types')

In [ ]:
oviendo.obs['celltype_tissue'] = np.nan 
oviendo.obs['celltype_tissue'] = oviendo.obs['Cell_Types'].astype(str) + '-' + oviendo.obs['Tissue_type'].astype(str)
counts = oviendo.obs['celltype_tissue'].value_counts()

counts

In [ ]:
toremove=['SOX9+ Epithelial-Fallopian Tube','SOX9+, LGR5+-Fallopian Tube','Glandular Epithelial-Fallopian Tube','Glandular Secretory-Fallopian Tube','EPCAM+-Uterine','OVGP1+ Secretory-Uterine','EPCAM+-Fallopian Tube']
oviendo = oviendo[~oviendo.obs['celltype_tissue'].isin(toremove)]

In [ ]:
oviendo = oviendo[oviendo.obs['menstrual_status'].isin(['Secretory']),:].copy()

In [ ]:
oviendo.obs['broadsecretorygroups']=np.nan 

broadgroupsdict = { 'Secretory Fallopian Tube' : ['OVGP1+ Secretory-Fallopian Tube'],
                   'Secretory Endometrial' : ['Glandular Epithelial-Uterine','Glandular Secretory-Uterine','Lumenal Epithelial-Uterine','SOX9+ Epithelial-Uterine']}

for i in broadgroupsdict.keys():
    ind = pd.Series(oviendo.obs.celltype_tissue).isin(broadgroupsdict[i])
    oviendo.obs.loc[ind,'broadsecretorygroups'] = i
                   
oviendo.obs['endometrialsecretorygroups']=np.nan 

endometrialgroupsdict = { 'Secretory Fallopian Tube' : ['OVGP1+ Secretory-Fallopian Tube'],
                   'Glandular Endometrial' : ['Glandular Epithelial-Uterine','Glandular Secretory-Uterine','SOX9+ Epithelial-Uterine'],
                    'Luminal Endometrial' : ['Lumenal Epithelial-Uterine'] }
for i in endometrialgroupsdict.keys():
    ind = pd.Series(oviendo.obs.celltype_tissue).isin(endometrialgroupsdict[i])
    oviendo.obs.loc[ind,'endometrialsecretorygroups'] = i

In [ ]:
sc.pl.umap(oviendo, color=['broadsecretorygroups','endometrialsecretorygroups'])

In [ ]:
glandularendo = oviendo[oviendo.obs['endometrialsecretorygroups'].isin(['Glandular Endometrial']),:].copy()
glandularendo = ad.AnnData.to_df(glandularendo)
glandularendo.to_csv(r'/home/j87832lw/mounting/oviduct/data/glandularendojune.csv')

In [ ]:
lumenalendo = oviendo[oviendo.obs['endometrialsecretorygroups'].isin(['Luminal Endometrial']),:].copy()
lumenalendo = ad.AnnData.to_df(lumenalendo)
lumenalendo.to_csv(r'/home/j87832lw/mounting/oviduct/data/lumenalendojune.csv')

In [ ]:
secretoryFT = oviendo[oviendo.obs['endometrialsecretorygroups'].isin(['Secretory Fallopian Tube']),:].copy()
secretoryFT = ad.AnnData.to_df(secretoryFT)
secretoryFT.to_csv(r'/home/j87832lw/mounting/oviduct/data/secretoryFTjune.csv')

In [ ]:
sc.pp.log1p(oviendo)
sc.tl.rank_genes_groups(oviendo, 'broadsecretorygroups', use_raw=False, groups=['Secretory Fallopian Tube'], reference='Secretory Endometrial', method='wilcoxon')
sc.pl.rank_genes_groups(oviendo, n_genes=25, sharey=False)

In [ ]:
gene_names = oviendo.uns['rank_genes_groups']['names'].tolist()

broadDEGs = [gene for sublist in gene_names for gene in sublist]

broadDEGs=broadDEGs[0:600]

with open(r'/home/j87832lw/mounting/oviduct/oviduct.all/broadDEGsjun.txt', 'w') as fp:
    for item in broadDEGs:
        fp.write("%s\n" % item)
    print('Done')